In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import Window
import pandas as pd
from pyspark.sql.functions import col

#### Lendo os dados

In [2]:
read_path = '/home/tatiane/Downloads/exemplo_linkage_cases-brazil-cities-time.csv/tmp/exemplo_linkage_cases-brazil-cities-time.csv'

In [3]:
df = spark.read.csv(read_path, sep = ",", multiLine = True)
# Base não apresenta cabeçalho

In [4]:
df.show()

+---+---+------------+------------+---------+---------+------+------+--------+--------+----+----+------------------+
|_c0|_c1|         _c2|         _c3|      _c4|      _c5|   _c6|   _c7|     _c8|     _c9|_c10|_c11|              _c12|
+---+---+------------+------------+---------+---------+------+------+--------+--------+----+----+------------------+
|  0|  0|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200225|20200225|   9|   9|               1.0|
|  2|  2|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200226|20200226|   9|   9|               1.0|
|  4|  4|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200227|20200227|   9|   9|               1.0|
|  6|  6|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200228|20200228|   9|   9|               1.0|
|  1|  1|       TOTAL|       TOTAL|     null|     null|     0|     0|20200225|20200225|   9|   9|0.7237569060773481|
|  5|  5|       TOTAL|       TOTAL|     null|     null|     0|  

#### Pegando informações do linkage

In [5]:
tot = df.count()
print('O número total de registros na base é de: {}'.format(tot))
print("Quantidade de colunas:", len(df.columns))

O número total de registros na base é de: 2248977
Quantidade de colunas: 13


#### Colocando cabeçalho nas colunas

In [6]:
df = df.select(col("_c0").alias("id1"), col("_c1").alias("id2"),col("_c2").alias("city1"),col("_c3").alias("city2"),col("_c4").alias("regiao_saude1"),col("_c5").alias("regiao_saude2"),col("_c6").alias("epi_week1"),col("_c7").alias("epi_week2"),col("_c8").alias("date1"),col("_c9").alias("date2"),col("_c10").alias("codmun1"),col("_c11").alias("codmun2"),col("_c12").alias("score"))
df.show()

+---+---+------------+------------+-------------+-------------+---------+---------+--------+--------+-------+-------+------------------+
|id1|id2|       city1|       city2|regiao_saude1|regiao_saude2|epi_week1|epi_week2|   date1|   date2|codmun1|codmun2|             score|
+---+---+------------+------------+-------------+-------------+---------+---------+--------+--------+-------+-------+------------------+
|  0|  0|São Paulo/SP|São Paulo/SP|    São Paulo|    São Paulo|   355030|   355030|20200225|20200225|      9|      9|               1.0|
|  2|  2|São Paulo/SP|São Paulo/SP|    São Paulo|    São Paulo|   355030|   355030|20200226|20200226|      9|      9|               1.0|
|  4|  4|São Paulo/SP|São Paulo/SP|    São Paulo|    São Paulo|   355030|   355030|20200227|20200227|      9|      9|               1.0|
|  6|  6|São Paulo/SP|São Paulo/SP|    São Paulo|    São Paulo|   355030|   355030|20200228|20200228|      9|      9|               1.0|
|  1|  1|       TOTAL|       TOTAL|      

In [7]:
df.printSchema()

root
 |-- id1: string (nullable = true)
 |-- id2: string (nullable = true)
 |-- city1: string (nullable = true)
 |-- city2: string (nullable = true)
 |-- regiao_saude1: string (nullable = true)
 |-- regiao_saude2: string (nullable = true)
 |-- epi_week1: string (nullable = true)
 |-- epi_week2: string (nullable = true)
 |-- date1: string (nullable = true)
 |-- date2: string (nullable = true)
 |-- codmun1: string (nullable = true)
 |-- codmun2: string (nullable = true)
 |-- score: string (nullable = true)



#### Criando colunas com comparação


In [8]:
df = df.withColumn('city', F.when(F.col('city1') == F.col('city2'), 1).otherwise(0))
df = df.withColumn('regiao_saude', F.when(F.col('regiao_saude1') == F.col('regiao_saude2'), 1).otherwise(0))
df = df.withColumn('date', F.when(F.col('date1') == F.col('date2'), 1).otherwise(0))
df = df.withColumn('epi_week', F.when(F.col('epi_week1') == F.col('epi_week2'), 1).otherwise(0))
df = df.withColumn('codmun', F.when(F.col('codmun1') == F.col('codmun2'), 1).otherwise(0))

In [10]:
df.limit(20).toPandas()

,id1,id2,city1,city2,regiao_saude1,regiao_saude2,epi_week1,epi_week2,date1,date2,codmun1,codmun2,score,city,regiao_saude,date,epi_week,codmun
0,0,0,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200225,20200225,9,9,1.0,1,1,1,1,1
1,2,2,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200226,20200226,9,9,1.0,1,1,1,1,1
2,4,4,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200227,20200227,9,9,1.0,1,1,1,1,1
3,6,6,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200228,20200228,9,9,1.0,1,1,1,1,1
4,1,1,TOTAL,TOTAL,None,None,0,0,20200225,20200225,9,9,0.7237569060773481,1,0,1,1,1
5,5,5,TOTAL,TOTAL,None,None,0,0,20200227,20200227,9,9,0.7237569060773481,1,0,1,1,1
6,3,3,TOTAL,TOTAL,None,None,0,0,20200226,20200226,9,9,0.7237569060773481,1,0,1,1,1
7,8,8,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200229,20200229,9,9,1.0,1,1,1,1,1
8,7,7,TOTAL,TOTAL,None,None,0,0,20200228,20200228,9,9,0.7237569060773481,1,0,1,1,1
9,10,10,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200301,20200301,10,10,1.0,1,1,1,1,1
